In [ ]:
import warnings
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['M1','M5','M15','H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]

    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])

    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['Time'].rank()

    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc

# Stats

In [14]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Hilo']].describe().round(0).astype(int)

Hilo                                   
     count mean  std min  25%  50%  75%   max
TF                                           
M5   99094  164  100   0  100  140  200  3500
M15  44862  297  178   0  180  255  370  6160
H1   10622  582  347  75  345  495  735  6160

In [15]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)

HL Pos                                   
     count mean  std min  25%  50%  75%   max
TF                                           
M5   51108  162  103   0   95  140  200  3500
M15  22806  294  183   0  175  250  365  6160
H1    5374  573  354  75  340  485  724  6160

In [16]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

HL Neg                                   
     count mean  std   min  25%  50%  75% max
TF                                           
M5   47986 -166   98 -2095 -205 -140 -100 -10
M15  22056 -301  173 -2675 -375 -260 -185 -40
H1    5248 -590  340 -3380 -745 -510 -350 -90

In [17]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Change']].describe().round(0).astype(int)

Change                                    
     count mean  std   min  25% 50%  75%   max
TF                                            
M5   99094    0  113 -2055  -55   0   55  2705
M15  44862    0  203 -2280 -100   0  100  3655
H1   10622   -2  402 -2875 -200   0  205  4280

In [18]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Abs']].describe().round(0).astype(int)

Chg Abs                                  
      count mean  std min 25%  50%  75%   max
TF                                           
M5    99094   79   81   0  25   55  105  2705
M15   44862  143  145   0  45  100  195  3655
H1    10622  284  284   0  90  205  385  4280

In [19]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Pos']].describe().round(0).astype(int)

Chg Pos                                  
      count mean  std min 25%  50%  75%   max
TF                                           
M5    51108   77   82   0  25   55  105  2705
M15   22806  140  147   0  45  100  190  3655
H1     5374  279  285   0  90  205  375  4280

In [20]:
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                  
      count mean  std   min  25%  50% 75% max
TF                                           
M5    47986  -82   80 -2055 -110  -60 -30  -5
M15   22056 -145  143 -2280 -195 -105 -50  -5
H1     5248 -289  282 -2875 -395 -205 -90  -5

# Snippets

In [21]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['T'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['Time'].rank()

    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc


Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Hilo']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['HL Pos']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['HL Neg']].describe().round(0).astype(int)

Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Change']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Abs']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Pos']].describe().round(0).astype(int)
Calc[Calc['T']!=1].groupby(['TF'], sort=0)[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                  
      count mean  std   min  25%  50% 75% max
TF                                           
M5    47986  -82   80 -2055 -110  -60 -30  -5
M15   22056 -145  143 -2280 -195 -105 -50  -5
H1     5248 -289  282 -2875 -395 -205 -90  -5